In [ ]:
%pip install lighteval==0.6.2
%pip install great-tables
%pip install polars

# Comparing different prompt formulations for the same task
In this notebook, we are going to use a very small model on a simple task, focusing on comparing several formulations for the input prompt, to see how they affect the results one can get.

In [2]:
import string
import os
from datetime import timedelta
from types import ModuleType
from ast import literal_eval

In [3]:
# For data viz
from great_tables import GT
import polars as pl
import polars.selectors as cs
from datasets import load_dataset

In [ ]:
# For evaluation
import lighteval
from lighteval.logging.evaluation_tracker import EvaluationTracker
from lighteval.models.model_config import BaseModelConfig, VLLMModelConfig
from lighteval.pipeline import ParallelismManager, Pipeline, PipelineParameters
from lighteval.metrics.metrics import Metrics
from lighteval.tasks.lighteval_task import LightevalTaskConfig, Doc
from lighteval.utils.utils import as_list, EnvConfig
from lighteval.utils.imports import is_accelerate_available, is_tgi_available

In [5]:
# Define for your usecase
cache_dir = "tmp"
max_samples = 10

## Comparing several formulations for the same task

Let's compare:
- using an MCQA evaluation (multi-choice question answer)
- using a generative evaluation

and for both, using variations of the same prompts.

We'll take the ARC dataset from AI2 for our experiments, using the challenge subset. You can first browse the dataset here: https://huggingface.co/datasets/allenai/ai2_arc?row=0.

### Let's define the core of our task logic

In [6]:
class ArcExplorationTask(LightevalTaskConfig):
    def __init__(self, name, prompt_function, metric):
        super().__init__(
            name=name,
            prompt_function=prompt_function,
            metric=as_list(metric),
            # It's a custom defined task
            suite=["custom"],
            # This defines our dataset and subsets
            hf_repo="allenai/ai2_arc",
            hf_subset="ARC-Challenge",
            hf_avail_splits=["train", "validation", "test"],
            evaluation_splits=["test"],
            # The few shot sample selection parameters
            few_shots_split="validation",
            few_shots_select="random", 
            # Other task parameters
            stop_sequence=[".", "\n"],
            generation_size=100,
        )

### Let's define our metrics

For a multichoice evaluation, we want the loglikelihood accuracy, normalized by length (= is the most probable choice the correct one?).

For a generative  evaluation, we want an exact match (= is the text generated matching the reference?).

In [7]:
metric_mcqa = Metrics.loglikelihood_acc_norm
metric_gen = Metrics.quasi_exact_match

### Let's define our prompt functions

A line of the ARC dataset is a dictionary, of the following shape
```python
{
    "question": "the prompt question",
    "choices": {
        "text": ["choice 1", "choice 2", ...],
        "label": ["A", "B", ...]
    },
    "answerKey": "the gold label"
}
```

Our prompt function will convert this to a template where we map all this info to our requested keys (query, choices, gold_index, an instruction if needed).


First, we define the most basic template possible. 
Prompt looks like this:
```
<the question?>
```
and we look at `<the choices>` directly

In [8]:
def arc_base(line, task_name: str = None):
    query= f"{line['question']}"
    choices=line["choices"]["text"]

    return Doc(
        task_name=task_name,
        query=query,
        choices=choices,
        gold_index=line["choices"]["label"].index(line["answerKey"]),
    )

We now add some context. Prompt looks like this:
```
Question: <the question?>
Answer: 
```
and we look at `<the choices>` directly too.

In [9]:
def arc_context(line, task_name: str = None):
    query= f"Question: {line['question']}"
    query += "\nAnswer: "
    choices=line["choices"]["text"]
    return Doc(
        task_name=task_name,
        query=query,
        choices=choices,
        gold_index=line["choices"]["label"].index(line["answerKey"]),
    )

We now add the choices to the prompt. Prompt looks like this:
```
Question: <the question?>
Choices:
A. <choice A>
B. <choice B>
...
Answer: 
```
and we look at `<the choices>` directly again.

In [10]:
letters = list(string.ascii_uppercase)

In [11]:
def arc_context_choices(line, task_name: str = None):
    query = f"Question: {line['question']}\n"
    query += "\n".join([f"{letters[ix]}. {choice}" for ix, choice in enumerate(line["choices"]["text"])])
    query += "\nAnswer: "
    choices=line["choices"]["text"]
    return Doc(
        task_name=task_name,
        query=query,
        choices=choices,
        gold_index=line["choices"]["label"].index(line["answerKey"]),
    )

Lastly, we do the same, but look at `<the choice labels>` instead.

In [12]:
def arc_context_labels(line, task_name: str = None):
    query = f"Question: {line['question']}\n"
    query += "\n".join([f"{letters[ix]}. {choice}" for ix, choice in enumerate(line["choices"]["text"])])
    query += "\nAnswer: "
    choices=[letters[ix] for ix in range(len(line["choices"]["text"]))]
    return Doc(
        task_name=task_name,
        query=query,
        choices=choices,
        gold_index=line["choices"]["label"].index(line["answerKey"]),
    )



### Let's now string everything together

In [13]:
task_module = ModuleType("task_module")
task_module.__file__ = ".",
task_module.TASKS_TABLE = [
    ArcExplorationTask(
        name="arc_base", 
        prompt_function=arc_base, 
        metric=[metric_mcqa, metric_gen]
    ),
    ArcExplorationTask(
        name="arc_context", 
        prompt_function=arc_context, 
        metric=[metric_mcqa, metric_gen]
    ),
    ArcExplorationTask(
        name="arc_context_choice", 
        prompt_function=arc_context_choices, 
        metric=[metric_mcqa, metric_gen]
    ),
    ArcExplorationTask(
        name="arc_context_labels", 
        prompt_function=arc_context_labels, 
        metric=[metric_mcqa, metric_gen]
    )
]

task_names = ["arc_base", "arc_context", "arc_context_choice", "arc_context_labels"]

# Let's launch our evaluation!

In [14]:
if is_accelerate_available():
    from accelerate import Accelerator, InitProcessGroupKwargs
    accelerator = Accelerator(kwargs_handlers=[InitProcessGroupKwargs(timeout=timedelta(seconds=3000))])
else:
    accelerator = None

In [15]:
# Parameters to save your evaluation results
evaluation_tracker = EvaluationTracker(
    output_dir=cache_dir,
    save_details=True,
    # These 2 options require you to be logged in with the huggingface_hub CLI
    #push_to_hub=True,
    #hub_results_org="your username", 
)


# Parameters of the overall pipeline
pipeline_params = PipelineParameters(
    launcher_type=ParallelismManager.ACCELERATE,
    env_config=EnvConfig(cache_dir=cache_dir),
    override_batch_size=1,
    max_samples=max_samples, # We only do a small run to test everything, remove to get actual results
    custom_tasks_directory=task_module # We can pass the path to a module or the module itself
)

# Model - here we use a VLLM, but we could be using TGI, Accelerate, etc
model_config = BaseModelConfig(
    pretrained="HuggingFaceTB/SmolLM-1.7B",
    dtype="bfloat16",
    use_chat_template=False,
)

tasks = ",".join([f"custom|{task}|3|0" for task in task_names])
# Now we're good to go!
pipeline = Pipeline(
    tasks=tasks,
    pipeline_parameters=pipeline_params,
    evaluation_tracker=evaluation_tracker,
    model_config=model_config,
)
pipeline.evaluate()
pipeline.save_and_push_results()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Splits: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [06:41<00:00, 401.95s/it]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [05:26<00:00,  8.17s/it]
1it [05:26, 326.90s/it]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [04:50<00:00,  7.25s/it]
2it [10:16, 305.21s/it]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [03:27<00:00,  5.19s/it]
3it [13:44, 260.60s/it]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
#pipeline.show_results()

In [17]:
results = pipeline.get_results()["results"]
results_processed = []
for eval_name, eval_results in results.items():
    results_processed.append({
        "Prompt function": (eval_name.split(":")[1] if ":" in eval_name else eval_name).replace("_", " "), 
        "Quasi Exact Match": eval_results["qem"], 
        "Normalized Accuracy": eval_results["acc_norm"]
    })
results_data = pl.from_dicts(results_processed, strict=False)
(GT(results_data.head(max_samples*4))
    .tab_header("Results")
     .tab_spanner(label="Evaluations", columns=["Quasi Exact Match", "Normalized Accuracy"])

)


GT(_tbl_data=shape: (5, 3)
┌────────────────────┬───────────────────┬─────────────────────┐
│ Prompt function    ┆ Quasi Exact Match ┆ Normalized Accuracy │
│ ---                ┆ ---               ┆ ---                 │
│ str                ┆ f64               ┆ f64                 │
╞════════════════════╪═══════════════════╪═════════════════════╡
│ arc base           ┆ 0.0               ┆ 0.3                 │
│ arc context        ┆ 0.1               ┆ 0.5                 │
│ arc context choice ┆ 0.4               ┆ 0.3                 │
│ arc context labels ┆ 0.0               ┆ 0.0                 │
│ all                ┆ 0.125             ┆ 0.275               │
└────────────────────┴───────────────────┴─────────────────────┘, _body=<great_tables._gt_data.Body object at 0x7f074da27010>, _boxhead=Boxhead([ColInfo(var='Prompt function', type=<ColInfoTypeEnum.default: 1>, column_label='Prompt function', column_align='left', column_width=None), ColInfo(var='Quasi Exact Match', type=<ColInfoTypeEnum.default: 1>, column_label='Quasi Exact Match', column_align='right', column_width=None), ColInfo(var='Normalized Accuracy', type=<ColInfoTypeEnum.default: 1>, column_label='Normalized Accuracy', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7f074da277c0>, _spanners=Spanners([SpannerInfo(spanner_id='Evaluations', spanner_level=0, spanner_label='Evaluations', spanner_units=None, spanner_pattern=None, vars=['Quasi Exact Match', 'Normalized Accuracy'], built=None)]), _heading=Heading(title='Results', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x7f074c8b6e30>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), tab

# Reading the results

In [18]:
path = f"{cache_dir}/details/HuggingFaceTB/SmolLM-1.7B/"

results = {}

for root, _, files in os.walk(path):
    for file in files:
        eval_name = file.split("|")[1]
        results[eval_name] = load_dataset("parquet", data_files=f"{root}/{file}")["train"]


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [19]:
# create a new DataFrame to store the transformed data
transformed_data = []
keys = ["example", "gold", "predictions", "metrics"]

# iterate over each dataset and its samples
for ix in range(max_samples * 2):
    for key in keys:
        cur_sample = {"Sample": f"Sample {ix}", "Type": key.capitalize()}
        for eval_name, df in sorted(results.items()):
            try:
                cur_result = literal_eval(results[eval_name][ix][key])
                if isinstance(cur_result, list):
                    if len(cur_result) == 1:
                        cur_sample[eval_name] = cur_result[0]
                    else:
                        cur_sample[eval_name] = "\n".join([str(i) for i in cur_result])
                elif isinstance(cur_result, dict):
                    for metric, value in cur_result.items():
                        cur_sample[eval_name] = str(value)
                        cur_sample["Type"] = f"{key.capitalize()}: {metric}"
            except SyntaxError:
                cur_sample[eval_name] = results[eval_name][ix][key]
                
        for k, v in cur_sample.items():
            # We replace new lines from python to markdown style for display in the table
            if isinstance(v, str):
                cur_sample[k] = v.replace("\n", "<br />")
        transformed_data.append(cur_sample)

### Let's look at the generative results

In [20]:
pl_data = pl.from_dicts(transformed_data, strict=False, infer_schema_length=200)

In [21]:
(GT(pl_data.head(max_samples*4))
    .tab_header("Comparing our different prompts' outputs")
    .tab_spanner(label="Samples", columns=cs.starts_with("arc"))
    .tab_stub(rowname_col="Type", groupname_col="Sample")
     .fmt_markdown(columns=cs.starts_with("arc"))
)

GT(_tbl_data=shape: (40, 6)
┌──────────┬──────────────┬──────────────────┬─────────────────┬─────────────────┬─────────────────┐
│ Sample   ┆ Type         ┆ arc_base         ┆ arc_context     ┆ arc_context_cho ┆ arc_context_lab │
│ ---      ┆ ---          ┆ ---              ┆ ---             ┆ ice             ┆ els             │
│ str      ┆ str          ┆ str              ┆ str             ┆ ---             ┆ ---             │
│          ┆              ┆                  ┆                 ┆ str             ┆ str             │
╞══════════╪══════════════╪══════════════════╪═════════════════╪═════════════════╪═════════════════╡
│ Sample 0 ┆ Example      ┆ Cities control   ┆ Question:       ┆ Question:       ┆ Question:       │
│          ┆              ┆ the amount of p… ┆ Cities control  ┆ Cities control  ┆ Cities control  │
│          ┆              ┆                  ┆ the a…          ┆ the a…          ┆ the a…          │
│ Sample 0 ┆ Gold         ┆ The air stays    ┆ The air stays   ┆ The air stays   ┆ A               │
│          ┆              ┆ cleaner.         ┆ cleaner.        ┆ cleaner.        ┆                 │
│ Sample 0 ┆ Predictions  ┆ It reduces the   ┆ 1               ┆  It becomes     ┆  D              │
│          ┆              ┆ amount of pollu… ┆                 ┆ safer to drive  ┆                 │
│          ┆              ┆                  ┆                 ┆ on …            ┆                 │
│ Sample 0 ┆ Metrics: qem ┆ 0                ┆ 0               ┆ 0               ┆ 0               │
│ Sample 1 ┆ Example      ┆ Which statement  ┆ Question: Which ┆ Question: Which ┆ Question: Which │
│          ┆              ┆ correctly desc…  ┆ statement corr… ┆ statement corr… ┆ statement corr… │
│ …        ┆ …            ┆ …                ┆ …               ┆ …               ┆ …               │
│ Sample 8 ┆ Metrics: qem ┆ 0                ┆ 0               ┆ 0               ┆ 0               │
│ Sample 9 ┆ Example      ┆ How much time is ┆ Question: How   ┆ Question: How   ┆ Question: How   │
│          ┆              ┆ required for …   ┆ much time is    ┆ much time is    ┆ much time is    │
│          ┆              ┆                  ┆ req…            ┆ req…            ┆ req…            │
│ Sample 9 ┆ Gold         ┆ 50 s             ┆ 50 s            ┆ 50 s            ┆ B               │
│ Sample 9 ┆ Predictions  ┆                  ┆ 100 m/2 m/s =   ┆ 100 s           ┆ 200 s           │
│          ┆              ┆                  ┆ 50 s            ┆                 ┆                 │
│ Sample 9 ┆ Metrics: qem ┆ 0                ┆ 0               ┆ 0               ┆ 0               │
└──────────┴──────────────┴──────────────────┴─────────────────┴─────────────────┴─────────────────┘, _body=<great_tables._gt_data.Body object at 0x7f074da26800>, _boxhead=Boxhead([ColInfo(var='Sample', type=<ColInfoTypeEnum.row_group: 3>, column_label='Sample', column_align='left', column_width=None), ColInfo(var='Type', type=<ColInfoTypeEnum.stub: 2>, column_label='Type', column_align='left', column_width=None), ColInfo(var='arc_base', type=<ColInfoTypeEnum.default: 1>, column_label='arc_base', column_align='left', column_width=None), ColInfo(var='arc_context', type=<ColInfoTypeEnum.default: 1>, column_label='arc_context', column_align='left', column_width=None), ColInfo(var='arc_context_choice', type=<ColInfoTypeEnum.default: 1>, column_label='arc_context_choice', column_align='left', column_width=None), ColInfo(var='arc_context_labels', type=<ColInfoTypeEnum.default: 1>, column_label='arc_context_labels', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7f074c181ea0>, _spanners=Spanners([SpannerInfo(spanner_id='Samples', spanner_level=0, spanner_label='Samples', spanner_units=None, spanner_pattern=None, vars=['arc_base', 'arc_context', 'arc_context_choice', 'arc_context_labels'], built=None)]), _heading=Heading(title="Comparing our different prompts' outputs", subtitle=None, prehea

We can observe that 
- the base format is too hard in generative mode: the model never predicts the correct ending
- base + question/answer tags format seems to incite the model to produce numbers, which is inadequate for a number of questions
- however (in the latter two) introducing the choices to the question helps the model predict a choice among the relevant ones!

Interestingly in the last case, when the choices are present but the model must predict the label, it is not systematically succeding in doing so. 

In some other cases, like samples 3 and 4, the model will not select the same choice when predicting the label or predicting the choice (if the choices were present).

### Let's look at the log probabilities output

In [22]:
(GT(pl_data.tail(max_samples * 4))
    .tab_header("Comparing our different prompts' outputs")
    .tab_spanner(label="Samples", columns=cs.starts_with("arc"))
    .tab_stub(rowname_col="Type", groupname_col="Sample")
     .fmt_markdown(columns=cs.starts_with("arc"))
)

GT(_tbl_data=shape: (40, 6)
┌───────────┬─────────────┬──────────────────┬─────────────────┬─────────────────┬─────────────────┐
│ Sample    ┆ Type        ┆ arc_base         ┆ arc_context     ┆ arc_context_cho ┆ arc_context_lab │
│ ---       ┆ ---         ┆ ---              ┆ ---             ┆ ice             ┆ els             │
│ str       ┆ str         ┆ str              ┆ str             ┆ ---             ┆ ---             │
│           ┆             ┆                  ┆                 ┆ str             ┆ str             │
╞═══════════╪═════════════╪══════════════════╪═════════════════╪═════════════════╪═════════════════╡
│ Sample 10 ┆ Example     ┆ Cities control   ┆ Question:       ┆ Question:       ┆ Question:       │
│           ┆             ┆ the amount of p… ┆ Cities control  ┆ Cities control  ┆ Cities control  │
│           ┆             ┆                  ┆ the a…          ┆ the a…          ┆ the a…          │
│ Sample 10 ┆ Gold        ┆ The air stays    ┆ The air stays   ┆ The air stays   ┆ A               │
│           ┆             ┆ cleaner.         ┆ cleaner.        ┆ cleaner.        ┆                 │
│ Sample 10 ┆ Predictions ┆ (-10.59571933746 ┆ (-9.65079307556 ┆ (-1.79188811779 ┆ (-1.44515538215 │
│           ┆             ┆ 3379, False)<b…  ┆ 1523,           ┆ 02222,          ┆ 6372,           │
│           ┆             ┆                  ┆ False)<br…      ┆ False)<b…       ┆ False)<br…      │
│ Sample 10 ┆ Metrics:    ┆ 1                ┆ 1               ┆ 0               ┆ 0               │
│           ┆ acc_norm    ┆                  ┆                 ┆                 ┆                 │
│ Sample 11 ┆ Example     ┆ Which statement  ┆ Question: Which ┆ Question: Which ┆ Question: Which │
│           ┆             ┆ correctly desc…  ┆ statement corr… ┆ statement corr… ┆ statement corr… │
│ …         ┆ …           ┆ …                ┆ …               ┆ …               ┆ …               │
│ Sample 18 ┆ Metrics:    ┆ 0                ┆ 0               ┆ 0               ┆ 0               │
│           ┆ acc_norm    ┆                  ┆                 ┆                 ┆                 │
│ Sample 19 ┆ Example     ┆ How much time is ┆ Question: How   ┆ Question: How   ┆ Question: How   │
│           ┆             ┆ required for …   ┆ much time is    ┆ much time is    ┆ much time is    │
│           ┆             ┆                  ┆ req…            ┆ req…            ┆ req…            │
│ Sample 19 ┆ Gold        ┆ 50 s             ┆ 50 s            ┆ 50 s            ┆ B               │
│ Sample 19 ┆ Predictions ┆ (-9.211626052856 ┆ (-7.63347101211 ┆ (-3.26651406288 ┆ (-1.73518586158 │
│           ┆             ┆ 445, False)<br…  ┆ 54785,          ┆ 14697,          ┆ 75244,          │
│           ┆             ┆                  ┆ False)<b…       ┆ False)<b…       ┆ False)<b…       │
│ Sample 19 ┆ Metrics:    ┆ 0                ┆ 0               ┆ 0               ┆ 0               │
│           ┆ acc_norm    ┆                  ┆                 ┆                 ┆                 │
└───────────┴─────────────┴──────────────────┴─────────────────┴─────────────────┴─────────────────┘, _body=<great_tables._gt_data.Body object at 0x7f074c8b4a30>, _boxhead=Boxhead([ColInfo(var='Sample', type=<ColInfoTypeEnum.row_group: 3>, column_label='Sample', column_align='left', column_width=None), ColInfo(var='Type', type=<ColInfoTypeEnum.stub: 2>, column_label='Type', column_align='left', column_width=None), ColInfo(var='arc_base', type=<ColInfoTypeEnum.default: 1>, column_label='arc_base', column_align='left', column_width=None), ColInfo(var='arc_context', type=<ColInfoTypeEnum.default: 1>, column_label='arc_context', column_align='left', column_width=None), ColInfo(var='arc_context_choice', type=<ColInfoTypeEnum.default: 1>, column_label='arc_context_choice', column_align='left', column_width=None), ColInfo(var='arc_context_labels', type=<ColInfoTypeEnum.default: 1>, column_label='arc_context_labels', column_align='left'

When looking at the log-likelihoods of continuations, what curiously seems to work best for this model is not having been provided with the examples in the prompt, in an opposite mechanism than for generative evaluations.